In [2]:
# python env: atom2_env_dev_v2

import json                                                                                                                             
import monty.json
import numpy as np
import pandas as pd
import traceback as tb
import sys
 
from pymatgen.core.structure import Structure
 
from atomate2.vasp.jobs.core import DielectricMaker
from atomate2.vasp.powerups import (
    update_user_incar_settings,
    update_user_potcar_functional,
    update_user_kpoints_settings
)
 
from fireworks import LaunchPad
from importlib import reload
 
from jobflow import Flow
from jobflow import run_locally
from jobflow.managers.fireworks import flow_to_workflow
 
from pathlib import Path
import pickle
from tqdm import tqdm

# LaunchPad

In [3]:
#   Paths to the candidates datasets
candidate_df_path    = Path("df_selected_v2.json")
 
#   Loading the candidates datasets
if not candidate_df_path.exists():
    raise Exception(f"The file containing the structures of the candidates was not found at {candidate_df_path}.")
else:
    df_selected = pd.read_json(candidate_df_path)

#   Iterating over the candidates to add them in the Launchpad 
for mpid, d in tqdm(zip(df_selected.index.values, df_selected['structure'])):

    # Initializes the MP structure
    struc = Structure.from_dict(d)
    structure = struc.get_primitive_structure()
 
    # Create a Dfpt dielectric WF to compute the dielectric tensor
    dielec_flow = DielectricMaker().make(structure=structure)
 
    # Power ups
        # Increase the kpoints density to 1500 per reciprocal atom for balance accuracy/complexity
    dielec_flow = update_user_kpoints_settings( dielec_flow, {"grid_density": 1500})
 
    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    dielec_flow.update_metadata({"Batch": "re2fractive_hg_v3", "mp-id": f"{mpid}"})
 
 
 
        # Specify:  the parallelization to optimize performance
        #           the electronic convergence criterion            (1E-5 eV otherwise by default),
        #           do not enforce a type of psp --> read in POTCAR (PBEsol otherwise by default)
        #           projection done in real space to improve efficiency (True otherwise)
    dielec_flow = update_user_incar_settings(   dielec_flow, {  "KPAR": 8,
                                                                "EDIFF": 1E-6,
                                                                "GGA": None,
                                                                "LREAL": 'False',
                                                                "ALGO": 'All',
                                                                "ISMEAR": 0,
                                                                "SIGMA": 0.03,
                                                                "LAECHG": 'False',
                                                                "LELF": 'False',
                                                                "LVTOT": 'False',
                                                                "LWAVE": 'False',
                                                                "PREC": 'Accurate',
                                                                "IBRION": -1,
                                                                "NSW": 0
                                                                })
 
        # Choose the type of PSP, here PBE_54
    dielec_flow = update_user_potcar_functional(dielec_flow, "PBE_54")
 
    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(dielec_flow)

    # submit the workflow to the FireWorks launchpad (requires a valid connection)
    lpad = LaunchPad.auto_load()
    lpad.add_wf(wf)
 
#   Final step: launch the calculations on a cluster using qlaunch 

0it [00:00, ?it/s]

2024-01-11 11:14:12,078 INFO Added a workflow. id_map: {-201: 8333}


1it [00:00,  4.71it/s]

2024-01-11 11:14:12,293 INFO Added a workflow. id_map: {-202: 8334}


2it [00:00,  4.69it/s]

...

198it [00:45,  4.42it/s]

2024-01-11 11:14:57,607 INFO Added a workflow. id_map: {-399: 8531}


199it [00:45,  4.53it/s]

2024-01-11 11:14:57,820 INFO Added a workflow. id_map: {-400: 8532}


200it [00:45,  4.35it/s]
